In [225]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp 

In [226]:
mp_holist = mp.solutions.holistic 
mp_draw = mp.solutions.drawing_utils

In [227]:
def mediapipe_detection(img, model):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img.flags.writeable = False                 
    result = model.process(img)                 # Make prediction
    img.flags.writeable = True                   
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) 
    return img, result

In [228]:
def draw_landmarks(img, result):
    mp_draw.draw_landmarks(img, result.face_landmarks, mp_holist.FACEMESH_CONTOURS) # Draw face connections
    mp_draw.draw_landmarks(img, result.pose_landmarks, mp_holist.POSE_CONNECTIONS) # Draw pose connections
    mp_draw.draw_landmarks(img, result.left_hand_landmarks, mp_holist.HAND_CONNECTIONS) # Draw left hand connections
    mp_draw.draw_landmarks(img, result.right_hand_landmarks, mp_holist.HAND_CONNECTIONS) # Draw right hand connections

In [229]:
def draw_styled_landmarks(img, result):
    mp_draw.draw_landmarks(img, result.face_landmarks, mp_holist.FACEMESH_CONTOURS, 
                             mp_draw.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), # color the joint 
                             mp_draw.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1) #color the connection
                             ) 
    # mp_draw.draw_landmarks(img, result.face_landmarks, mp_holist.FACEMESH_CONTOURS, 
    #                          mp_draw.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), # color the joint 
    #                          mp_draw.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1) #color the connection
    #                          ) 
    
    mp_draw.draw_landmarks(img, result.pose_landmarks, mp_holist.POSE_CONNECTIONS,
                             mp_draw.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_draw.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    mp_draw.draw_landmarks(img, result.left_hand_landmarks, mp_holist.HAND_CONNECTIONS, 
                             mp_draw.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_draw.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    mp_draw.draw_landmarks(img, result.right_hand_landmarks, mp_holist.HAND_CONNECTIONS, 
                             mp_draw.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_draw.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [230]:
mp_holist.POSE_CONNECTIONS

frozenset({(0, 1),
           (0, 4),
           (1, 2),
           (2, 3),
           (3, 7),
           (4, 5),
           (5, 6),
           (6, 8),
           (9, 10),
           (11, 12),
           (11, 13),
           (11, 23),
           (12, 14),
           (12, 24),
           (13, 15),
           (14, 16),
           (15, 17),
           (15, 19),
           (15, 21),
           (16, 18),
           (16, 20),
           (16, 22),
           (17, 19),
           (18, 20),
           (23, 24),
           (23, 25),
           (24, 26),
           (25, 27),
           (26, 28),
           (27, 29),
           (27, 31),
           (28, 30),
           (28, 32),
           (29, 31),
           (30, 32)})

In [231]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holist.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        draw_styled_landmarks(image, results)

        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

C:\Users\araya\AppData\Roaming\Python\Python312\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>


In [232]:
cap.release()
cv2.destroyAllWindows()

In [233]:
results.pose_landmarks.landmark[0].visibility

0.9995303153991699

In [234]:
len(results.pose_landmarks.landmark)

33

In [235]:
def extract_keypoints(results):
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    left_hnd=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    right_hnd=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    return np.concatenate([pose,left_hnd,right_hnd,face])
# concatenating for the model to detect the sign language

In [236]:
extract_keypoints(results).shape

(1662,)

In [237]:
import os 
video_dir = "C:/Users/araya/Desktop/keypoints/video_extract"
video_list = []
video_list = os.listdir(video_dir)

len(video_list)

21

In [238]:
video_list

['กฎกระทรวง.mp4',
 'กฎหมายรัฐธรรมนูญ.mp4',
 'กรมอนามัย.mp4',
 'กรรม.mp4',
 'กรรมสิทธิ์.mp4',
 'กระโดด.mp4',
 'กล้วยบวชชี.mp4',
 'กล้วยเชื่อม.mp4',
 'เขิน.mp4',
 'เขื่อนดิน.mp4',
 'เขื่อนสิริกิติ์.mp4',
 'เข้าใจผิด.mp4',
 'เคย.mp4',
 'เครียด.mp4',
 'เครื่องปั่นดิน.mp4',
 'เครื่องหมายการค้า.mp4',
 'เจอ.mp4',
 'เจ้าหนี้.mp4',
 'เช่าซื้อ.mp4',
 'เช่าทรัพย์.mp4',
 'โจทก์.mp4']

In [239]:
# Path for exported data, numpy arrays
Model_Data=os.path.join('Data for different actions')

actions = np.array(video_list)

no_of_seqs = 1

# 30 frames in length
seq_length = 160

In [240]:
actions

array(['กฎกระทรวง.mp4', 'กฎหมายรัฐธรรมนูญ.mp4', 'กรมอนามัย.mp4',
       'กรรม.mp4', 'กรรมสิทธิ์.mp4', 'กระโดด.mp4', 'กล้วยบวชชี.mp4',
       'กล้วยเชื่อม.mp4', 'เขิน.mp4', 'เขื่อนดิน.mp4',
       'เขื่อนสิริกิติ์.mp4', 'เข้าใจผิด.mp4', 'เคย.mp4', 'เครียด.mp4',
       'เครื่องปั่นดิน.mp4', 'เครื่องหมายการค้า.mp4', 'เจอ.mp4',
       'เจ้าหนี้.mp4', 'เช่าซื้อ.mp4', 'เช่าทรัพย์.mp4', 'โจทก์.mp4'],
      dtype='<U21')

In [241]:
# just creating the folders and sub folders

for action in actions: 
    try: 
        os.makedirs(os.path.join(Model_Data, action))
    except:
        pass

In [242]:
# import os
# augment_dir = "C:/Users/araya/Desktop/augments"

# augment_list = []
# augment_list = os.listdir(augment_dir)
# augment_list

In [243]:
# actions = list(actions)

In [244]:
# for x in augment_list:
#     # print(x)
#     actions.append(x)
# actions

In [245]:
# actions = np.array(actions)
# actions

Collecting keypoint values for Training nd Testing

In [246]:
# Define the directory where your videos are stored
directory = "C:/Users/araya/Desktop/keypoints/video_extract"

In [247]:
directory

'C:/Users/araya/Desktop/keypoints/video_extract'

In [248]:
txt = "hello, my name is Peter, I am 26 years old"

x = txt.split(", ")

print(x)

['hello', 'my name is Peter', 'I am 26 years old']


In [249]:
for filename in actions:
    print(directory + '/' + filename)

C:/Users/araya/Desktop/keypoints/video_extract/กฎกระทรวง.mp4
C:/Users/araya/Desktop/keypoints/video_extract/กฎหมายรัฐธรรมนูญ.mp4
C:/Users/araya/Desktop/keypoints/video_extract/กรมอนามัย.mp4
C:/Users/araya/Desktop/keypoints/video_extract/กรรม.mp4
C:/Users/araya/Desktop/keypoints/video_extract/กรรมสิทธิ์.mp4
C:/Users/araya/Desktop/keypoints/video_extract/กระโดด.mp4
C:/Users/araya/Desktop/keypoints/video_extract/กล้วยบวชชี.mp4
C:/Users/araya/Desktop/keypoints/video_extract/กล้วยเชื่อม.mp4
C:/Users/araya/Desktop/keypoints/video_extract/เขิน.mp4
C:/Users/araya/Desktop/keypoints/video_extract/เขื่อนดิน.mp4
C:/Users/araya/Desktop/keypoints/video_extract/เขื่อนสิริกิติ์.mp4
C:/Users/araya/Desktop/keypoints/video_extract/เข้าใจผิด.mp4
C:/Users/araya/Desktop/keypoints/video_extract/เคย.mp4
C:/Users/araya/Desktop/keypoints/video_extract/เครียด.mp4
C:/Users/araya/Desktop/keypoints/video_extract/เครื่องปั่นดิน.mp4
C:/Users/araya/Desktop/keypoints/video_extract/เครื่องหมายการค้า.mp4
C:/Users/araya/D

In [250]:
# # Set mediapipe model 
# for action in actions:
#     video_path = os.path.join("C:/Users/araya/Desktop/keypoints/video_extract", action)
#     cap = cv2.VideoCapture(video_path)
#     cap.set(cv2.CAP_PROP_FPS, 60)
#     length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#     print("LENGTH:" + str(length))
#     # keypoints = []

#     if not cap.isOpened():
#         print(f"Error opening video file: {video_path}")
#         continue

#     with mp_holist.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#         for seq in range(no_of_seqs):
#             for frame_num in range(seq_length):

#                 ret, frame = cap.read()
#                 if not ret:
#                     print(f"End of video {video_path}")
#                     break
                
#                 img, results = mediapipe_detection(frame, holistic)
#                 draw_styled_landmarks(img, results)

#                 # print(frame_num)

#                 if frame_num == 0: 
#                     cv2.putText(img, 'DATA COLLECTION STARTED', (120,200), 
#                                 cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
#                     cv2.putText(img, f'Collecting frames for - {action} Sequence Number - {seq}', (15,12), 
#                                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
#                     cv2.imshow('OpenCV Window', img)
#                     cv2.waitKey(2000)  # 2 seconds delay for setup
#                 else: 
#                     cv2.putText(img, f'Collecting frames for - {action} Sequence Number - {seq}', (15,12), 
#                                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
#                     cv2.imshow('OpenCV Window', img)

#                 keypoints = extract_keypoints(results)
#                 # keypoints.append(results)
#                 npy_path = os.path.join(Model_Data, action, f"frame_{frame_num}.npy")
#                 os.makedirs(os.path.dirname(npy_path), exist_ok=True)
#                 np.save(npy_path, keypoints)

#                 if cv2.waitKey(1) & 0xFF == ord('q'):
#                     break

#             if not ret:
#                 break

#     cap.release()
#     cv2.destroyAllWindows()

In [251]:
# import numpy as np 

# X = [[1,2,3]]
# X.append([6,8,10])
# X.append([20,9,4])
# X

In [252]:
# Set mediapipe model 
for action in actions:
    video_path = os.path.join("C:/Users/araya/Desktop/keypoints/video_extract", action)
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_FPS, 60)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print("LENGTH:" + str(length))
    keypoints = []

    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        continue

    with mp_holist.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for seq in range(no_of_seqs):
            for frame_num in range(seq_length):

                ret, frame = cap.read()
                if not ret:
                    print(f"End of video {video_path}")
                    break
                
                img, results = mediapipe_detection(frame, holistic)
                draw_styled_landmarks(img, results)

                # print(frame_num)

                if frame_num == 0: 
                    cv2.putText(img, 'DATA COLLECTION STARTED', (120,200), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(img, f'Collecting frames for - {action} Sequence Number - {seq}', (15,12), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Window', img)
                    cv2.waitKey(2000)  # 2 seconds delay for setup
                else: 
                    cv2.putText(img, f'Collecting frames for - {action} Sequence Number - {seq}', (15,12), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Window', img)

                x = extract_keypoints(results)
                keypoints.append(x)
                npy_path = os.path.join(Model_Data, action, f"{action.split(".")[0]}.npy")
                os.makedirs(os.path.dirname(npy_path), exist_ok=True)
                np.save(npy_path, keypoints)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            if not ret:
                break

    cap.release()
    cv2.destroyAllWindows()

LENGTH:142
End of video C:/Users/araya/Desktop/keypoints/video_extract\กฎกระทรวง.mp4
LENGTH:134
End of video C:/Users/araya/Desktop/keypoints/video_extract\กฎหมายรัฐธรรมนูญ.mp4
LENGTH:151
End of video C:/Users/araya/Desktop/keypoints/video_extract\กรมอนามัย.mp4
LENGTH:100
End of video C:/Users/araya/Desktop/keypoints/video_extract\กรรม.mp4
LENGTH:184
LENGTH:94
End of video C:/Users/araya/Desktop/keypoints/video_extract\กระโดด.mp4
LENGTH:182
LENGTH:128
End of video C:/Users/araya/Desktop/keypoints/video_extract\กล้วยเชื่อม.mp4
LENGTH:100
End of video C:/Users/araya/Desktop/keypoints/video_extract\เขิน.mp4
LENGTH:161
LENGTH:208
LENGTH:105
End of video C:/Users/araya/Desktop/keypoints/video_extract\เข้าใจผิด.mp4
LENGTH:82
End of video C:/Users/araya/Desktop/keypoints/video_extract\เคย.mp4
LENGTH:118
End of video C:/Users/araya/Desktop/keypoints/video_extract\เครียด.mp4
LENGTH:234
LENGTH:126
End of video C:/Users/araya/Desktop/keypoints/video_extract\เครื่องหมายการค้า.mp4
LENGTH:92
End of 

In [253]:
cap.release()
cv2.destroyAllWindows()

In [254]:
file_paths = []
for action in actions:
    video_path = os.path.join('Data for different actions/', action)
    # print(video_path)
    # print(action)
    file_paths.append(video_path + '/' + action.split(".")[0] + ".npy")
print(file_paths)

['Data for different actions/กฎกระทรวง.mp4/กฎกระทรวง.npy', 'Data for different actions/กฎหมายรัฐธรรมนูญ.mp4/กฎหมายรัฐธรรมนูญ.npy', 'Data for different actions/กรมอนามัย.mp4/กรมอนามัย.npy', 'Data for different actions/กรรม.mp4/กรรม.npy', 'Data for different actions/กรรมสิทธิ์.mp4/กรรมสิทธิ์.npy', 'Data for different actions/กระโดด.mp4/กระโดด.npy', 'Data for different actions/กล้วยบวชชี.mp4/กล้วยบวชชี.npy', 'Data for different actions/กล้วยเชื่อม.mp4/กล้วยเชื่อม.npy', 'Data for different actions/เขิน.mp4/เขิน.npy', 'Data for different actions/เขื่อนดิน.mp4/เขื่อนดิน.npy', 'Data for different actions/เขื่อนสิริกิติ์.mp4/เขื่อนสิริกิติ์.npy', 'Data for different actions/เข้าใจผิด.mp4/เข้าใจผิด.npy', 'Data for different actions/เคย.mp4/เคย.npy', 'Data for different actions/เครียด.mp4/เครียด.npy', 'Data for different actions/เครื่องปั่นดิน.mp4/เครื่องปั่นดิน.npy', 'Data for different actions/เครื่องหมายการค้า.mp4/เครื่องหมายการค้า.npy', 'Data for different actions/เจอ.mp4/เจอ.npy', 'Data for

In [255]:
def load_keypoint_sequences(file_paths):
    keypoint_sequences = []
    for file_path in file_paths:
        keypoints = np.load(file_path)
        keypoint_sequences.append(torch.tensor(keypoints, dtype=torch.float32))
    return keypoint_sequences

In [256]:
# Load the sequences
import torch
sequences = load_keypoint_sequences(file_paths)
sequences

[tensor([[ 0.5013,  0.2452, -1.2167,  ...,  0.5663,  0.2188,  0.0098],
         [ 0.4997,  0.2482, -1.4690,  ...,  0.5652,  0.2181,  0.0106],
         [ 0.4984,  0.2500, -1.4853,  ...,  0.5654,  0.2185,  0.0112],
         ...,
         [ 0.4861,  0.2513, -1.3416,  ...,  0.5572,  0.2177,  0.0091],
         [ 0.4873,  0.2514, -1.3574,  ...,  0.5575,  0.2172,  0.0097],
         [ 0.4883,  0.2516, -1.3579,  ...,  0.5577,  0.2170,  0.0101]]),
 tensor([[ 0.4922,  0.2382, -1.2850,  ...,  0.5578,  0.2124,  0.0094],
         [ 0.4920,  0.2405, -1.4288,  ...,  0.5571,  0.2116,  0.0099],
         [ 0.4920,  0.2409, -1.4093,  ...,  0.5567,  0.2122,  0.0098],
         ...,
         [ 0.4814,  0.2260, -1.3318,  ...,  0.5503,  0.1923,  0.0123],
         [ 0.4815,  0.2257, -1.3351,  ...,  0.5503,  0.1921,  0.0122],
         [ 0.4815,  0.2255, -1.3497,  ...,  0.5501,  0.1919,  0.0124]]),
 tensor([[ 0.5049,  0.2371, -1.2115,  ...,  0.5643,  0.2082,  0.0088],
         [ 0.5045,  0.2381, -1.1896,  ...,  0

In [257]:
# Pad the sequences to the same length
padded_sequences = pad_sequence(sequences, batch_first=True)
pad_sequence
print(padded_sequences.shape) # (batch_size, max_sequence_length, num_keypoints)

torch.Size([21, 160, 1662])


In [258]:
labels = [action.split(".")[0] for action in actions]
labels

['กฎกระทรวง',
 'กฎหมายรัฐธรรมนูญ',
 'กรมอนามัย',
 'กรรม',
 'กรรมสิทธิ์',
 'กระโดด',
 'กล้วยบวชชี',
 'กล้วยเชื่อม',
 'เขิน',
 'เขื่อนดิน',
 'เขื่อนสิริกิติ์',
 'เข้าใจผิด',
 'เคย',
 'เครียด',
 'เครื่องปั่นดิน',
 'เครื่องหมายการค้า',
 'เจอ',
 'เจ้าหนี้',
 'เช่าซื้อ',
 'เช่าทรัพย์',
 'โจทก์']

In [259]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

labels = le.fit_transform(labels)
labels

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20], dtype=int64)

In [260]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

# Create a custom dataset
class KeypointDataset(Dataset):
    def __init__(self, file_paths, labels):
        self.file_paths = file_paths
        self.labels = labels
    
    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, idx):
        keypoints = np.load(self.file_paths[idx])
        label = self.labels[idx]
        return torch.tensor(keypoints, dtype=torch.float32), label

In [261]:
# Create the dataset
dataset = KeypointDataset(file_paths, labels)

In [262]:
print(dataset.file_paths)
print(dataset.labels)

['Data for different actions/กฎกระทรวง.mp4/กฎกระทรวง.npy', 'Data for different actions/กฎหมายรัฐธรรมนูญ.mp4/กฎหมายรัฐธรรมนูญ.npy', 'Data for different actions/กรมอนามัย.mp4/กรมอนามัย.npy', 'Data for different actions/กรรม.mp4/กรรม.npy', 'Data for different actions/กรรมสิทธิ์.mp4/กรรมสิทธิ์.npy', 'Data for different actions/กระโดด.mp4/กระโดด.npy', 'Data for different actions/กล้วยบวชชี.mp4/กล้วยบวชชี.npy', 'Data for different actions/กล้วยเชื่อม.mp4/กล้วยเชื่อม.npy', 'Data for different actions/เขิน.mp4/เขิน.npy', 'Data for different actions/เขื่อนดิน.mp4/เขื่อนดิน.npy', 'Data for different actions/เขื่อนสิริกิติ์.mp4/เขื่อนสิริกิติ์.npy', 'Data for different actions/เข้าใจผิด.mp4/เข้าใจผิด.npy', 'Data for different actions/เคย.mp4/เคย.npy', 'Data for different actions/เครียด.mp4/เครียด.npy', 'Data for different actions/เครื่องปั่นดิน.mp4/เครื่องปั่นดิน.npy', 'Data for different actions/เครื่องหมายการค้า.mp4/เครื่องหมายการค้า.npy', 'Data for different actions/เจอ.mp4/เจอ.npy', 'Data for

In [263]:
# Collate function for padding
def collate_fn(batch):
    sequences, labels = zip(*batch)
    padded_sequences = pad_sequence(sequences, batch_first=True)
    return padded_sequences, torch.tensor(labels)

In [264]:
# Create the DataLoader
batch_size = 4
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
data_loader

In [265]:
import torch
import torch.nn as nn
import torch.optim as optim

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate the LSTM
        out, _ = self.lstm(x, (h0, c0))
        
        # Use the last time step's output for classification
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [266]:
# Set device (use GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [267]:
# Initialize the model, loss function, and optimizer
model = LSTMModel(input_size=1662, hidden_size=128, num_layers=2, num_classes=21).to(device)

In [268]:
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [269]:
# Training loop
num_epochs = 300
for epoch in range(num_epochs):
    model.train()
    for i, (sequences, labels) in enumerate(data_loader):
        # Move data to the device
        sequences = sequences.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/300], Loss: 3.0771
Epoch [2/300], Loss: 3.3151
Epoch [3/300], Loss: 3.3754
Epoch [4/300], Loss: 3.4476
Epoch [5/300], Loss: 3.1946
Epoch [6/300], Loss: 3.5937
Epoch [7/300], Loss: 3.7086
Epoch [8/300], Loss: 3.6550
Epoch [9/300], Loss: 3.8139
Epoch [10/300], Loss: 3.6697
Epoch [11/300], Loss: 3.2579
Epoch [12/300], Loss: 3.5836
Epoch [13/300], Loss: 3.2831
Epoch [14/300], Loss: 3.5168
Epoch [15/300], Loss: 3.3247
Epoch [16/300], Loss: 3.4316
Epoch [17/300], Loss: 1.4669
Epoch [18/300], Loss: 3.1141
Epoch [19/300], Loss: 3.3620
Epoch [20/300], Loss: 0.9862
Epoch [21/300], Loss: 3.3801
Epoch [22/300], Loss: 2.8123
Epoch [23/300], Loss: 3.5468
Epoch [24/300], Loss: 3.2391
Epoch [25/300], Loss: 3.2749
Epoch [26/300], Loss: 0.9542
Epoch [27/300], Loss: 3.0362
Epoch [28/300], Loss: 1.3972
Epoch [29/300], Loss: 3.0906
Epoch [30/300], Loss: 1.5403
Epoch [31/300], Loss: 3.1929
Epoch [32/300], Loss: 2.8340
Epoch [33/300], Loss: 2.5325
Epoch [34/300], Loss: 2.5039
Epoch [35/300], Loss: 3

In [270]:
pad_sequence(sequences, batch_first=True)

tensor([[[ 0.4922,  0.2382, -1.2850,  ...,  0.5578,  0.2124,  0.0094],
         [ 0.4920,  0.2405, -1.4288,  ...,  0.5571,  0.2116,  0.0099],
         [ 0.4920,  0.2409, -1.4093,  ...,  0.5567,  0.2122,  0.0098],
         ...,
         [ 0.4814,  0.2260, -1.3318,  ...,  0.5503,  0.1923,  0.0123],
         [ 0.4815,  0.2257, -1.3351,  ...,  0.5503,  0.1921,  0.0122],
         [ 0.4815,  0.2255, -1.3497,  ...,  0.5501,  0.1919,  0.0124]]])

In [271]:
# Put the model in evaluation mode
model.eval()

# No need to track gradients during inference
with torch.no_grad():
    # Get the model's output (logits)
    outputs = model(padded_sequences)

# outputs = torch.softmax(outputs, dim=1)
# outputs = torch.max(outputs,1)

outputs


tensor([[ 4.5624,  0.7731, -1.6153, -2.3025, -3.1507,  0.1362, -3.7282,  0.6829,
         -2.6290, -1.6117, -3.3710, -2.5167, -2.1433,  3.0694, -2.7982, -0.6838,
         -0.1604,  3.6548, -0.0283,  2.6791,  2.9549],
        [-1.6816,  5.0027, -2.4236, -1.5547, -4.9270,  2.0412, -6.1473,  3.3839,
         -1.5182, -1.0241, -3.2922, -1.4616, -0.2065,  2.6639, -4.4712,  1.2923,
          1.1402,  2.6263,  1.5590, -1.0941,  0.4534],
        [-1.4900,  0.2579,  6.3374,  2.1396, -3.5190, -1.9748, -1.7491,  0.6330,
          1.5378, -2.4932, -0.8810,  3.0957, -2.5056, -0.6579, -3.3464,  2.3156,
         -1.5508, -1.5359,  0.1149, -1.3826, -1.7413],
        [-4.7755, -3.3709, -0.0426,  5.7503, -2.4058,  1.0891, -3.0297, -1.0608,
          5.8777, -3.7441, -1.9000,  5.7858,  1.2945, -1.7067, -2.2699,  1.4590,
          1.2112, -2.0696, -1.0037, -0.7496, -1.3273],
        [-0.7007, -0.8241, -1.6710, -1.2503,  6.1529, -2.3028,  1.0994, -0.4096,
         -2.0354, -0.7682,  1.7312, -1.1438, -2.274

In [272]:
padded_sequences

tensor([[[ 0.5013,  0.2452, -1.2167,  ...,  0.5663,  0.2188,  0.0098],
         [ 0.4997,  0.2482, -1.4690,  ...,  0.5652,  0.2181,  0.0106],
         [ 0.4984,  0.2500, -1.4853,  ...,  0.5654,  0.2185,  0.0112],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.4922,  0.2382, -1.2850,  ...,  0.5578,  0.2124,  0.0094],
         [ 0.4920,  0.2405, -1.4288,  ...,  0.5571,  0.2116,  0.0099],
         [ 0.4920,  0.2409, -1.4093,  ...,  0.5567,  0.2122,  0.0098],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.5049,  0.2371, -1.2115,  ...,  0.5643,  0.2082,  0.0088],
         [ 0.5045,  0.2381, -1.1896,  ...,  0

In [273]:
len(padded_sequences)

21

In [321]:
file_paths = ["Data for different actions/กล้วยบวชชี_2.mp4/กล้วยบวชชี_2.npy"]

In [322]:
# Load the sequences
import torch
sequences = load_keypoint_sequences(file_paths)
# Change list to numpy array 
sequences = np.array(sequences)
# Change numpy array to tensor
sequences = torch.FloatTensor(sequences)
sequences = pad_sequence(sequences, batch_first=True)
sequences

tensor([[[ 0.5319,  0.2821, -1.6797,  ...,  0.6169,  0.2514,  0.0109],
         [ 0.5346,  0.2813, -1.7538,  ...,  0.6164,  0.2506,  0.0119],
         [ 0.5365,  0.2812, -1.7556,  ...,  0.6163,  0.2507,  0.0109],
         ...,
         [ 0.5340,  0.2852, -1.2054,  ...,  0.6196,  0.2422,  0.0160],
         [ 0.5345,  0.2806, -1.4284,  ...,  0.6202,  0.2413,  0.0160],
         [ 0.5356,  0.2758, -1.3837,  ...,  0.6207,  0.2407,  0.0163]]])

In [323]:
outputs = model(sequences)
outputs

tensor([[ 2.7387, -1.2895, -0.3022, -1.4025, -1.3495, -0.6996,  4.4123, -1.6544,
         -1.6072, -0.0755, -1.7109, -1.6890, -1.0813,  1.1921, -1.0088, -1.3502,
         -0.4197,  1.3098, -2.9438,  4.7029,  1.8431]],
       grad_fn=<AddmmBackward0>)

In [287]:
labels = [action.split(".")[0] for action in actions]
labels

['กฎกระทรวง',
 'กฎหมายรัฐธรรมนูญ',
 'กรมอนามัย',
 'กรรม',
 'กรรมสิทธิ์',
 'กระโดด',
 'กล้วยบวชชี',
 'กล้วยเชื่อม',
 'เขิน',
 'เขื่อนดิน',
 'เขื่อนสิริกิติ์',
 'เข้าใจผิด',
 'เคย',
 'เครียด',
 'เครื่องปั่นดิน',
 'เครื่องหมายการค้า',
 'เจอ',
 'เจ้าหนี้',
 'เช่าซื้อ',
 'เช่าทรัพย์',
 'โจทก์']

### -------------------------------------------------------------------------------------------------------------------------------------------- ###

In [ ]:
from torch.nn.utils.rnn import pad_sequence
import torch
# Load the sequences
import numpy as np 
import os
Model_Data=os.path.join('Data for different actions')
action = "กฎหมายรัฐธรรมนูญ.mp4"
a = np.load(os.path.join(Model_Data, action, "กฎหมายรัฐธรรมนูญ.npy"))
a = torch.from_numpy(a)

# Pad the sequences to the same length
padded_sequences = pad_sequence(a, batch_first=True)
len(padded_sequences)

In [803]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

In [804]:
labelMap = {label:num for num, label in enumerate(actions)}

In [ ]:
labelMap

In [806]:
seqs, labels = [], []

for action in actions:
    for seq in range(no_of_seqs):
        window = []
        for frame_num in range(seq_length):
            res = np.load(os.path.join(Model_Data, action, f"frame_{frame_num}.npy")) 
            window.append(res)
        seqs.append(window)

        labels.append(labelMap[action])

In [ ]:
np.array(seqs).shape

In [809]:
X_data = np.array(seqs)

In [ ]:
X_data.shape

In [ ]:
labels

In [ ]:
# changing the labels from 0,1,2 to categorical data for easier accessebility
Y_label = to_categorical(labels).astype(int)
Y_label

In [ ]:
Y_label.shape

In [ ]:
# splitting
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_label, test_size=0.3)
X_test.shape

### Building LSTM

In [815]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [816]:
# adding the logs folder
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
# neural network

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))

# adding 64 units for dense layer
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
# eg
eg_res = [.7, 0.2, 0.1]
actions[np.argmax(eg_res)]

In [819]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, Y_train, epochs=300, callbacks=[tb_callback])
# tensorboard --logdir=.

In [ ]:
model.summary()

In [ ]:
res=model.predict(X_test)

In [ ]:
# again the actions with the max value provided by softmax is returned
actions[np.argmax(res[4])]

In [ ]:
actions[np.argmax(Y_test[4])]

In [ ]:
res[0]

In [ ]:
Y_test[0]

### Evaluate

In [836]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
Y_hat = model.predict(X_train)

In [838]:
Y_true = np.argmax(Y_train, axis=1).tolist()
# one hot encoding
Y_hat = np.argmax(Y_hat, axis=1).tolist()

In [ ]:
Y_hat

In [ ]:
# confution matrix
multilabel_confusion_matrix(Y_true, Y_hat)

In [ ]:
accuracy_score(Y_true, Y_hat)